# Article Title Generator (V2)

Summarization use-case in which the user provides an article, which the LLM will analyze to suggest an SEO-optimized title.

**NOTES**:

1. This version supports website scrapping using Selenium (based on the code from **/week1/community-
   contributions/day1-webscraping-selenium-for-javascript.ipynb** - Thanks for the contribution!)
2. Leverage streaming (OpenAI only).
3. The following models were configured:\
   
    a. OpenAI gpt-4o-mini\
    b. Llama llama3.2\
    c. Deepseek deepseek-r1:1.5b\

   It is possible to configure additional models by adding the new model to the MODELS dictionary and its
   initialization to the CLIENTS dictionary. Then, call the model with --> ***answer =
   get_answer('NEW_MODEL')***.
5. Improved system_prompt to provide specific SEO best practices to adopt during the title generation.
6. Rephrased the system_prompt to ensure the model provides a single Title (not a list of suggestions).
7. Includes function to remove unrequired thinking/reasoning verbose from the model response (Deepseek). 
8. Users are encouraged to assess and rank the suggested titles using any headline analyzer tool online.
   Example: https://www.isitwp.com/headline-analyzer/. 

In [0]:
# install required libraries if they were not part of the requirements.txt
!pip install selenium
!pip install undetected-chromedriver

In [0]:
# confirming Llama is loaded
!ollama pull llama3.2

In [0]:
# imports
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup

In [0]:
# set environment variables for OpenAi
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# validate API Key
if not api_key:
    raise ValueError("No API key was found! Please check the .env file.")

In [0]:
# constants
MODELS = { 'GPT': 'gpt-4o-mini', 
          'LLAMA': 'llama3.2', 
          'DEEPSEEK': 'deepseek-r1:1.5b'
         }

CLIENTS = { 'GPT': OpenAI(), 
            'LLAMA': OpenAI(base_url='http://localhost:11434/v1', api_key='ollama'),
            'DEEPSEEK': OpenAI(base_url='http://localhost:11434/v1', api_key='ollama') 
          }

# path to Chrome
CHROME_PATH = "C:/Program Files/Google/Chrome/Application/chrome.exe"

**Webcrawler** (based on the code from __/week1/community-contributions/day1-webscraping-selenium-for-javascript.ipynb__)

In [0]:
class WebsiteCrawler:
    def __init__(self, url, wait_time=20, chrome_path=None):
        """
        Initialize the WebsiteCrawler using Selenium to scrape JavaScript-rendered content.
        """
        self.url = url
        self.wait_time = wait_time

        options = uc.ChromeOptions()
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("--disable-blink-features=AutomationControlled")
        # options.add_argument("--headless=new") # For Chrome >= 109 - unreliable on my end!
        options.add_argument("start-maximized")
        options.add_argument(
            "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
        )
        if chrome_path:
            options.binary_location = chrome_path

        self.driver = uc.Chrome(options=options)

        try:
            # Load the URL
            self.driver.get(url)

            # Wait for Cloudflare or similar checks
            time.sleep(10)

            # Ensure the main content is loaded
            WebDriverWait(self.driver, self.wait_time).until(
                EC.presence_of_element_located((By.TAG_NAME, "main"))
            )

            # Extract the main content
            main_content = self.driver.find_element(By.CSS_SELECTOR, "main").get_attribute("outerHTML")

            # Parse with BeautifulSoup
            soup = BeautifulSoup(main_content, "html.parser")
            self.title = self.driver.title if self.driver.title else "No title found"
            self.text = soup.get_text(separator="\n", strip=True)

        except Exception as e:
            print(f"Error occurred: {e}")
            self.title = "Error occurred"
            self.text = ""

        finally:
            self.driver.quit()


### Prompts

In [0]:
# system prompt
system_prompt = """
    You are an experienced SEO-focused copywriter. The user will provide an article, and your task is to analyze its content and generate a single, most effective, keyword-optimized title to maximize SEO performance.

Instructions:
Ignore irrelevant content, such as the current title (if any), navigation menus, advertisements, or unrelated text.
Prioritize SEO best practices, considering:
Keyword relevance and search intent (informational, transactional, etc.).
Readability and engagement.
Avoiding keyword stuffing.
Ensure conciseness and clarity, keeping the title under 60 characters when possible for optimal SERP display.
Use a compelling structure that balances informativeness and engagement, leveraging formats like:
Listicles ("10 Best Strategies for…")
How-to guides ("How to Boost…")
Questions ("What Is the Best Way to…")
Power words to enhance click-through rates (e.g., "Proven," "Ultimate," "Essential").
Provide only one single, best title—do not suggest multiple options.
Limit the answer to the following Response Format (Markdown):
Optimized Title: [Provide only one title here]
Justification: [Explain why this title is effective for SEO]

    """

**Provide the article URL and get its content for analysis**

In [0]:
# article url - change to any other article URL
article_url = "https://searchengineland.com/seo-trends-2025-447745"

# get article content
article = WebsiteCrawler(url=article_url, chrome_path=CHROME_PATH)

In [0]:
# user prompt
user_prompt = """
Following the article to be analyzed to suggest a title. Limit the answer to the following Response Format (Markdown): 
Optimized Title: [Provide only one title here]
Justification: [Explain why this title is effective for SEO].
"""

user_prompt = f"{user_prompt} {article}"
                

In [0]:
# message list
messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
           ]

In [0]:
# get suggested title
def get_title(model, **kwargs):
    # stream if GPT
    if 'stream' in kwargs:
        response = CLIENTS[model].chat.completions.create(
                model=MODELS[model],
                messages=messages,
                stream=kwargs['stream']
            )
    else:
        response = CLIENTS[model].chat.completions.create(
                model=MODELS[model],
                messages=messages,
            )

    return response
    

In [0]:
# filter response from model verbose - like Deepseek reasoning/thinking verbose
def filter_response(response):
    # Find last occurrence of 'Optimized Title:' to avoid displaying reasoning verbose
    substring = 'Optimized Title:'
    start = response.rfind('Optimized Title:')
    if start > -1:
        filtered_response = response[start:]

    # insert line break to preserve format
    filtered_response = filtered_response.replace("**Justification:**", "\n**Justification:**")
    
    return filtered_response

In [0]:
# display suggested title
def display_title(model):
    # get model-suggested title
    title = get_title(model)
    
    display(Markdown(f"### {model} (___{MODELS[model]}___) Answer\n\n_______")) 

    response = ""

    if model == 'GPT':
        display_handle = display(Markdown(""), display_id=True)
        # for chunk in stream:
        for chunk in get_title(model=model, stream=True):
            response += chunk.choices[0].delta.content or ''
            response = (
                response.replace("```","")
                .replace("markdown", "")
                .replace("Optimized Title:", "**Optimized Title:**")
                .replace("Justification:", "**Justification:**")
            )
            update_display(Markdown(response), display_id=display_handle.display_id)
    else:
        response = get_title(model=model)
        response = response.choices[0].message.content
        response = filter_response(response)
        response = (
            response.replace("Optimized Title:", "**Optimized Title:**")
            .replace("Justification:", "**Justification:**")
        )
        display(Markdown(response))

### Get OpenAI Suggested Title

In [0]:
# get and display openAi suggested title
display_title(model='GPT')

### Get Llama Suggested Title

In [0]:
# get and display Llama suggested title
display_title(model='LLAMA')

### Get Deepseek Suggested Title

In [0]:
# get and display Deepseek title
display_title(model='DEEPSEEK')

### Observations

1. **Selenium:** The headless option (__options.add_argument("--headless=new")__), while ideal to speed up the scanning process, presented problems while scanning several websites (including openai.com and canva.com).
2. **Deepseek challenges:**\
   a.It always returns its thinking/reasoning verbose, which, while helpful to understand how it works, is not always
      required, such as in this example code. A new function (**filter_response**) was created to remove the additional verbose.\
   b. It is unreliable with the response, sometimes returning the required format for the response instead of the
      actual response. For example, for the title, it may sometimes return:
   
   **Optimized Title:** \[The user wants the suggested title here]
    
### Suggested future improvements

1. Add the logic that would allow each model to assess the recommendations from the different models and 
   select the best among these.
2. Add the logic to leverage an API (if available) that automatically assesses the suggested titles.